In [1]:
import numpy as np
import re
import time
from stanfordcorenlp import StanfordCoreNLP
import torch
from torch.autograd import Function
from torch.autograd import Variable
import tensorly as tl
from tensorly.decomposition import parafac
from tensorly.decomposition import tucker
from tqdm import tqdm
import time
import math
import logging
from sklearn.decomposition import PCA
from torch.utils.data import DataLoader
from sklearn import preprocessing
from random import shuffle
BATCH_SIZE = 128
tl.set_backend('pytorch')
help(tl.set_backend)

Using numpy backend.


Help on function set_backend in module tensorly:

set_backend(backend_name)
    Sets the backend for TensorLy
    
        The backend will be set as specified and operations will used that backend
    
    Parameters
    ----------
    backend_name : {'mxnet', 'numpy', 'pytorch', 'tensorflow', 'cupy'}, default is 'numpy'



Using pytorch backend.


In [2]:
b = np.fromfile('tensors_2010.dat',dtype=np.float64).reshape((8000,-1,200))
Y = np.fromfile('tensors_2010_labels.dat',dtype=np.int)
position_tag_1 = np.fromfile('tensors_2010_entity1.dat',dtype=np.int).reshape((-1,1))
position_tag_2 = np.fromfile('tensors_2010_entity2.dat',dtype=np.int).reshape((-1,1))
position_tag = np.concatenate((position_tag_1,position_tag_2),axis =1)
b.shape,Y.shape,position_tag_1.shape,position_tag_2.shape,position_tag.shape

((8000, 85, 200), (8000,), (8000, 1), (8000, 1), (8000, 2))

In [3]:
position_tag.shape

(8000, 2)

In [4]:
temp = [i for i in range(85)]
c = []
for i in range(8000):
    position_a = (np.array(temp) - position_tag[i][0]).reshape((-1,1))
    position_b = (np.array(temp) - position_tag[i][1]).reshape((-1,1))
    position_tag_now = np.concatenate((position_a,position_b),axis = 1)
    c.append(np.concatenate((b[i],position_tag_now),axis = 1))
c = np.stack(c)
b = c
b.shape

(8000, 85, 202)

In [5]:
inputs = []
for i in range(8000):
    inputs.append((torch.from_numpy(b[i]),Y[i],position_tag[i].astype(np.int64)))
shuffle(inputs)
flag = int(len(inputs)*0.7)
train = inputs[:flag]
test = inputs[flag:]

In [6]:
t_e = []
t_y = []
t_p = []

for E,Y,pos in test:
    t_e.append(E)
    t_y.append(Y)
    t_p.append(pos)
t_e = torch.stack(t_e)
t_p = torch.from_numpy(np.stack(t_p))
t_y = torch.from_numpy(np.array(t_y))
t_e.shape, t_y.shape, t_p.shape

(torch.Size([2400, 85, 202]), torch.Size([2400]), torch.Size([2400, 2]))

# 0.6608333333333334 3轮

In [ ]:
##### import torch.nn as nn
import torchvision
import matplotlib.pyplot as plt
import torch.nn.functional as F
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(  # input shape (1, 53, 200)
            nn.Conv2d(
                in_channels=1,      # input height
                out_channels=85,    # n_filters
                kernel_size=(1,202),      # filter size
            ),      # output shape (16, 53, 200)
            nn.ReLU(),    # activation
        )
        self.maxpool = nn.MaxPool2d(kernel_size=(85,1))   # 在 2x2 空间里向下采样, output shape (16, 53, 1)
        self.output = nn.Linear(255, 10)   # fully connected layer, output 10 classes

    def forward(self, x, pos):
        torch.set_default_dtype(torch.double)
        x = x.reshape((-1,1,85,202)).double()
        x = self.conv1(x.double())
        
        x1 = torch.zeros(x.shape)
        x2 = torch.zeros(x.shape)
        x3 = torch.zeros(x.shape)
        
        min_x1 = torch.ones(x.shape) * (-1000)
        min_x2 = torch.ones(x.shape) * (-1000)
        min_x3 = torch.ones(x.shape) * (-1000)
        
        for i in range(pos.shape[0]):
            x1[i, :, :pos[i][0], :]
            min_x1[i, :, :pos[i][0], :]  = 0.0
            x2[i, :, pos[i][0]:pos[i][1], :]  = 1.0
            min_x2[i, :, pos[i][0]:pos[i][1], :] = 0.0
            x3[i, :, pos[i][1]:, :] = 1.0
            min_x3[i, :, pos[i][1]:, :] = 0.0
            
         # 卷积结果*mask得到分段后的卷积向量
        x1 *= x
        x2 *= x
        x3 *= x
        
        # 将无用部分赋予最小值，避免影响maxpool操作
        x1 += min_x1
        x2 += min_x2
        x3 += min_x3
        
        # 分段池化
        x1 = self.maxpool(x1)
        x2 = self.maxpool(x2)
        x3 = self.maxpool(x3)
        
        # 展平
        x1 = x1.view(x1.size(0), -1)
        x2 = x2.view(x2.size(0), -1)
        x3 = x3.view(x3.size(0), -1)
        
        x = torch.cat((x1,x2,x3),dim=1)

        output = self.output(x)
        return output

In [ ]:
import torch.optim.lr_scheduler as lr_scheduler
# 创建数据加载器
loader = DataLoader(
    dataset = train, 
    batch_size = BATCH_SIZE,      # mini batch size
    shuffle = True,               # 设置随机洗牌
    num_workers = 2,              # 加载数据的进程个数
)
net = CNN()
optimizer = torch.optim.Adam(net.parameters(),lr=0.01)
loss_F = F.cross_entropy
print(net)
start = time.time()
for epoch in range(1,201): 
    for step, (batch_x, batch_y,pos) in enumerate(loader):
        pred = net(batch_x,pos)
        loss = loss_F(pred,batch_y) # 计算loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if epoch % 1 == 0 and step == 2: # 每50步，计算精度
            with torch.no_grad():
                test_pred = net(t_e,t_p)
                prob = torch.nn.functional.softmax(test_pred,dim=1)
                pred_cls = torch.argmax(prob,dim=1)
                acc = (pred_cls.int() == t_y.int()).sum().numpy() / pred_cls.size()[0]
                print(f"{epoch}: accuracy:{acc} loss: {loss} time: {time.time() - start}")

CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 85, kernel_size=(1, 202), stride=(1, 1))
    (1): ReLU()
  )
  (maxpool): MaxPool2d(kernel_size=(85, 1), stride=(85, 1), padding=0, dilation=1, ceil_mode=False)
  (output): Linear(in_features=255, out_features=10, bias=True)
)
1: accuracy:0.17041666666666666 loss: 38.51878670936104 time: 2.446072816848755
2: accuracy:0.6104166666666667 loss: 6.373206883226785 time: 9.893219709396362
3: accuracy:0.6608333333333334 loss: 46.95810607836431 time: 17.177539825439453
4: accuracy:0.625 loss: 15.07727171985417 time: 24.447836875915527
5: accuracy:0.64125 loss: 45.81822458598132 time: 31.73485279083252
6: accuracy:0.6241666666666666 loss: 27.440087151773756 time: 38.885934829711914
7: accuracy:0.6354166666666666 loss: 62.72835027941746 time: 46.34409689903259
8: accuracy:0.6070833333333333 loss: 14.905748365720104 time: 54.02220797538757
9: accuracy:0.6216666666666667 loss: 34.397942902604655 time: 61.19856572151184
10: accuracy:0.63125 loss: 16.13

104: accuracy:0.6191666666666666 loss: 3.2954997007950126e-06 time: 769.0078718662262
105: accuracy:0.6208333333333333 loss: 1.0200174038743626e-14 time: 776.6239819526672
106: accuracy:0.61875 loss: 24.1119405228546 time: 784.6740329265594
107: accuracy:0.6241666666666666 loss: 2.5942655046407026e-07 time: 792.5353949069977
108: accuracy:0.6125 loss: 0.48848585291950125 time: 800.0485377311707
109: accuracy:0.6008333333333333 loss: 31.945406165961643 time: 807.8726189136505
110: accuracy:0.6058333333333333 loss: 3.8396946360982205 time: 815.1402349472046
111: accuracy:0.6154166666666666 loss: 1.146333685379958 time: 822.5600459575653
112: accuracy:0.61375 loss: 0.4681683272732505 time: 830.1075267791748
113: accuracy:0.6216666666666667 loss: 0.10944453089896644 time: 837.3411557674408
114: accuracy:0.61625 loss: 0.00024917584387509906 time: 844.7750589847565
115: accuracy:0.6129166666666667 loss: 0.16875865218413644 time: 852.3665940761566
116: accuracy:0.6216666666666667 loss: 0.0086